# [WIP] End-to-end example

Let's put together a few of the techniques outlined in this section and show how to perform retrieval after we've generated our query(ies) by building a Q&A bot over the LangChain YouTube videos.

## Setup
#### Install dependencies

In [ ]:
# %pip install -qU langchain langchain-community langchain-openai youtube-transcript-api pytube elasticsearch

#### Set environment variables

We'll use OpenAI in this example:

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

# Optional, uncomment to trace runs with LangSmith. Sign up here: https://smith.langchain.com.
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

### Set up integrations

We'll use Elasticsearch for our vectorstore. We can run an Elasticsearch instance locally with Docker:

```bash
docker run -p 9200:9200 -e "discovery.type=single-node" -e "xpack.security.enabled=false" -e "xpack.security.http.ssl.enabled=false" docker.elastic.co/elasticsearch/elasticsearch:8.9.0
```

### Load documents

We can use the `YouTubeLoader` to load transcripts of a few LangChain videos:

In [2]:
from langchain_community.document_loaders import YoutubeLoader

urls = [
    "https://www.youtube.com/watch?v=pbAd8O1Lvm4",
    "https://www.youtube.com/watch?v=ylrew7qb8sQ",
    "https://www.youtube.com/watch?v=uRya4zRrRx4",
    "https://www.youtube.com/watch?v=hvAPnpSfSGo",
    "https://www.youtube.com/watch?v=ZcEMLz27sL4",
    "https://www.youtube.com/watch?v=3wAON0Lqviw",
    "https://www.youtube.com/watch?v=jx7xuHlfsEQ",
    "https://www.youtube.com/watch?v=xn1jEjRyJ2U",
    "https://www.youtube.com/watch?v=SaDzIVkYqyY",
    "https://www.youtube.com/watch?v=gqhlqdawHT4",
    "https://www.youtube.com/watch?v=Ce03oEotdPs",
    "https://www.youtube.com/watch?v=rZus0JtRqXE",
    "https://www.youtube.com/watch?v=HAn9vnJy6S4",
    "https://www.youtube.com/watch?v=dA1cHGACXCo",
    "https://www.youtube.com/watch?v=ZcEMLz27sL4",
    "https://www.youtube.com/watch?v=hvAPnpSfSGo",
    "https://www.youtube.com/watch?v=EhlPDL4QrWY",
    "https://www.youtube.com/watch?v=mmBo8nlu2j0",
    "https://www.youtube.com/watch?v=rQdibOsL1ps",
    "https://www.youtube.com/watch?v=28lC4fqukoc",
    "https://www.youtube.com/watch?v=es-9MgxB-uc",
    "https://www.youtube.com/watch?v=wLRHwKuKvOE",
    "https://www.youtube.com/watch?v=ObIltMaRJvY",
    "https://www.youtube.com/watch?v=DjuXACWYkkU",
    "https://www.youtube.com/watch?v=o7C9ld6Ln-M",
]
docs = []
for url in urls:
    docs.extend(YoutubeLoader.from_youtube_url(url, add_video_info=True).load())

Here are the titles of the videos we've loaded:

In [3]:
[doc.metadata["title"] for doc in docs]

['Self-reflective RAG with LangGraph: Self-RAG and CRAG',
 'WebVoyager',
 'LangGraph: Planning Agents',
 'LangGraph: Multi-Agent Workflows',
 'Streaming Events: Introducing a new `stream_events` method',
 'LangSmith: In-Depth Platform Overview',
 'LangSmith in 10 Minutes',
 'RAG from scratch: Part 8 (Query Translation -- Step Back)',
 'RAG from scratch: Part 9 (Query Translation -- HyDE)',
 'RAG from scratch: Part 7 (Query Translation -- Decomposition - v1)',
 'LangChain Agents with Open Source Models!',
 'Gemini + Google Retrieval Agent from a LangChain Template',
 'OpenGPTs',
 'Building a web RAG chatbot: using LangChain, Exa (prev. Metaphor), LangSmith, and Hosted Langserve',
 'Streaming Events: Introducing a new `stream_events` method',
 'LangGraph: Multi-Agent Workflows',
 'Build and Deploy a RAG app with Pinecone Serverless',
 'Auto-Prompt Builder (with Hosted LangServe)',
 'Build a Full Stack RAG App With TypeScript',
 'Getting Started with Multi-Modal LLMs',
 'SQL Research Assi

Here's the metadata associated with each video. We can see that each document also has a title, view count, publication date, and length:

In [5]:
docs[0].metadata

{'source': 'pbAd8O1Lvm4',
 'title': 'Self-reflective RAG with LangGraph: Self-RAG and CRAG',
 'description': 'Unknown',
 'view_count': 7946,
 'thumbnail_url': 'https://i.ytimg.com/vi/pbAd8O1Lvm4/hq720.jpg',
 'publish_date': '2024-02-07 00:00:00',
 'length': 1058,
 'author': 'LangChain'}

And here's a sample from a document's contents:

In [4]:
docs[0].page_content[:500]

"hi this is Lance from Lang chain I'm going to be talking about using Lang graph to build a diverse and sophisticated rag flows so just to set the stage the basic rag flow you can see here starts with a question retrieval of relevant documents from an index which are passed into the context window of an llm for generation of an answer grounded in the ret documents so that's kind of the basic outline and we can see it's like a very linear path um in practice though you often encounter a few differ"

### Indexing documents

Whenever we perform retrieval we need to create an index of documents that we can query. We'll use a vector store to index our documents, and we'll chunk them first to make our retrievals more concise and precise:

In [ ]:
import datetime

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma, ElasticsearchStore
from langchain_openai import OpenAIEmbeddings

# clean up metadata
for doc in docs:
    doc.metadata["publish_date"] = datetime.datetime.strptime(
        doc.metadata["publish_date"], "%Y-%m-%d %H:%M:%S"
    ).strftime("%Y-%m-%d")


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000, chunk_overlap=500, add_start_index=True
)
chunked_docs = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = ElasticsearchStore.from_documents(
    chunked_docs,
    embeddings,
    index_name="langchain_youtube_2",
    es_url="http://localhost:9200",
)

## Retrieval without query analysis

We can perform similarity search on a user question directly to find chunks relevant to the question:

In [6]:
search_results = vectorstore.similarity_search("how do I build a RAG agent")
print(search_results[0].metadata["title"])
print(search_results[0].page_content[:500])

OpenGPTs
it decides to use a tool and so importantly it lets us know that it's deciding to use a tool and then it also lets us know what the result of the tool is and then it starts streaming back the response so this is streaming not just tokens but also these intermediate steps which provide really good visibility into what is going on we can see here we can see the response that we got back from tavil um and then we can see um the response from the AI and so there's lots of dad jokes in here this is u


This works pretty well! Our first result is quite relevant to the question.


What if we wanted to search for results from a specific time period?

In [10]:
search_results = vectorstore.similarity_search("videos on RAG published in 2023")
print(search_results[0].metadata["title"])
print(search_results[0].metadata["publish_date"])
print(search_results[0].page_content[:500])

OpenGPTs
2024-01-31 00:00:00
it decides to use a tool and so importantly it lets us know that it's deciding to use a tool and then it also lets us know what the result of the tool is and then it starts streaming back the response so this is streaming not just tokens but also these intermediate steps which provide really good visibility into what is going on we can see here we can see the response that we got back from tavil um and then we can see um the response from the AI and so there's lots of dad jokes in here this is u


Our first result is from 2024, and not very relevant to the input. Since we're just searching against document contents, there's no way for the results to be filtered on any document attributes.

What if we wanted to know about deploying a LangChain chain as a REST API?

In [11]:
search_results = vectorstore.similarity_search("chain as rest api")
print(search_results[0].metadata["title"])
print(search_results[0].page_content[1500:2000])

Building a web RAG chatbot: using LangChain, Exa (prev. Metaphor), LangSmith, and Hosted Langserve
 further and see um that the inputs to each of those kind of Lambda steps is going to be one of those documents and then we're outputting um like the highlights DL segment and then formatting that with our prompt template uh if you recall from when we were constructing it um so now we have kind of our uh fully constructed chain uh for our uh search enable chatbot with XF um and now let's convert that to Lang serve um so to do that we'll go back to vs code um and here we're going to um start with


This brings up LangServe, the package for deploying chains as REST API's, as desired.

But what if we added that we wanted a chain that made use of multi-modal models?

In [86]:
search_results = vectorstore.similarity_search(
    "how to use multi-modal models in a chain and turn chain into a rest api"
)
print(search_results[0].metadata["title"])
print(search_results[0].page_content[:500])

Streaming Events: Introducing a new `stream_events` method
streaming is uh an incredibly important ux consideration for building L Ms in a few ways first of all even if you're just working with a single llm call it can often take a while and you might want to stream individual tokens to the user so they can see what's happening as the llm responds second of all a lot of the things that we build in the laying chain are more complicated chains or agents and so being able to stream the intermediate steps what tool are being called what the input to those t


Our first result ends up not being about LangServe or multi-modal models. In reality "chains as rest API" and "using multi-modal models" are two fairly distinct questions that should be queried for separately.

## Query analysis

To handle these failure modes we'll do some query structuring and decomposition. First we'll define a **query schema** and use a function-calling model to convert a user question into a structured queries. The structured nature of the query schema allows us to do query structuring and routing, and the fact that we can extract multiple of these allows us to do decomposition and expansion.

### Query schema
In this case we'll have explicit min and max attributes for publication date so that it can be filtered on. And we'll add separate attributes for searches against the transcript contents versus the video title.

In [21]:
import datetime
from typing import Literal, Optional, Tuple

from langchain_core.pydantic_v1 import BaseModel, Field


class TutorialSearch(BaseModel):
    """Search over a database of tutorial videos about a software library."""

    content_search: str = Field(
        ...,
        description="Similarity search query applied to video transcripts.",
    )
    earliest_publish_date: Optional[datetime.date] = Field(
        None, description="Earliest publish date filter, inclusive."
    )
    latest_publish_date: Optional[datetime.date] = Field(
        None, description="Latest publish date filter, exclusive."
    )

    def pretty_print(self) -> None:
        for field in self.__fields__:
            if getattr(self, field) is not None and getattr(self, field) != getattr(
                self.__fields__[field], "default", None
            ):
                print(f"{field}: {getattr(self, field)}")

### Query generation

To convert user questions to structured queries we'll make use of OpenAI's function-calling API. Since the latest OpenAI models can return multiple function invocations each turn, this approach automatically supports query expansion and decomposition.

In [22]:
from langchain.output_parsers import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

system = """You are an expert at converting user questions into database queries. \
You have access to a database of tutorial videos about a software library for building LLM-powered applications. \
Given a question, return a list of database queries optimized to retrieve the most relevant results.

Perform query expansion. If there are multiple common ways of phrasing a user question \
or common synonyms for key words in the question, make sure to return multiple versions \
of the query with the different phrasings.

Perform query decomposition. If the user input contains a multi-part question, make \
sure to return a separate query for each distinct sub-question.

If there are acronyms or words you are not familiar with, do not try to rephrase them."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("examples", optional=True),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{question}"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm_with_tools = llm.bind_tools([TutorialSearch])
query_analyzer = (
    {"question": RunnablePassthrough()}
    | prompt
    | llm_with_tools
    | PydanticToolsParser(tools=[TutorialSearch])
)

Let's see what queries our analyzer generates for the questions we searched earlier:

In [23]:
for query in query_analyzer.invoke("videos on RAG published in 2023"):
    query.pretty_print()
    print()

content_search: RAG
earliest_publish_date: 2023-01-01
latest_publish_date: 2024-01-01



In [24]:
for query in query_analyzer.invoke(
    "how to use multi-modal models in a chain and turn chain into a rest api"
):
    query.pretty_print()
    print()

content_search: multi-modal models in a chain

content_search: turn chain into a REST API



### Improvements: Adding examples to the prompt

To tune our results we can add some examples of inputs questions and gold standard output queries to our prompt. We'll focus on examples that show how to route and expand queries, to either be against titles or content, how to structure them with filters, and how to decompose them:

In [25]:
examples = []

In [26]:
question = "What is Web Voyager? How about Gemini?"
queries = [
    TutorialSearch(
        content_search="what is Web Voyager",
        title_search="Web Voyager",
    ),
    TutorialSearch(content_search="What is Gemini", title_search="Gemini"),
]
examples.append({"input": question, "tool_calls": queries})

In [27]:
question = "Have they released any chat langchain updates since 2024?"
queries = [
    TutorialSearch(
        title_search="chat langchain",
        content_search="chat langchain",
        earliest_publish_date=datetime.date(2024, 1, 1),
    ),
]
examples.append({"input": question, "tool_calls": queries})

In [28]:
question = "How to build multi-agent system and stream intermediate steps from it"
queries = [
    TutorialSearch(
        content_search="How to build multi-agent system",
        title_search="multi-agent system",
    ),
    TutorialSearch(
        content_search="how to stream intermediate steps from multi-agent system",
        title_search="stream intermediate steps multi-agent system",
    ),
    TutorialSearch(
        content_search="how to stream intermediate steps",
        title_search="stream intermediate steps",
    ),
]
examples.append({"input": question, "tool_calls": queries})

Now we need to update our prompt template and chain so that the examples are included in each prompt. Since we're working with OpenAI function-calling, we'll need to do a bit of extra structuring to send example inputs and outputs to the model. We'll create a `tool_example_to_messages` helper function to handle this for us:

In [29]:
import uuid
from typing import Dict, List

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
)


def tool_example_to_messages(example: Dict) -> List[BaseMessage]:
    messages: List[BaseMessage] = [HumanMessage(content=example["input"])]
    openai_tool_calls = []
    for tool_call in example["tool_calls"]:
        openai_tool_calls.append(
            {
                "id": str(uuid.uuid4()),
                "type": "function",
                "function": {
                    "name": tool_call.__class__.__name__,
                    "arguments": tool_call.json(),
                },
            }
        )
    messages.append(
        AIMessage(content="", additional_kwargs={"tool_calls": openai_tool_calls})
    )
    tool_outputs = example.get("tool_outputs") or [
        "This is an example of a correct usage of this tool. Well done. Make sure to continue using the tool this way."
    ] * len(openai_tool_calls)
    for output, tool_call in zip(tool_outputs, openai_tool_calls):
        messages.append(ToolMessage(content=output, tool_call_id=tool_call["id"]))
    return messages


example_msgs = [msg for ex in examples for msg in tool_example_to_messages(ex)]
query_analyzer_with_examples = (
    {"question": RunnablePassthrough()}
    | prompt.partial(examples=example_msgs)
    | llm_with_tools
    | PydanticToolsParser(tools=[TutorialSearch])
)

In [30]:
for query in query_analyzer_with_examples.invoke(
    "how to use multi-modal models in a chain and turn chain into a rest api"
):
    query.pretty_print()
    print()

content_search: how to use multi-modal models in a chain

content_search: how to turn chain into a REST API



In [31]:
for query in query_analyzer_with_examples.invoke(
    "How to do extraction with agent? How to build agent with anthropic"
):
    query.pretty_print()
    print()

content_search: How to do extraction with agent

content_search: How to build agent with anthropic



## Retrieval with query analysis

Our query analysis looks pretty good; now let's try using our generated queries to actually perform retrieval. We'll define a custom retrieval lambda that takes our output queries and correctly applies them to our vector store

In [58]:
from typing import List

from langchain.chains.query_constructor.ir import (
    Comparator,
    Comparison,
    Operation,
    Operator,
    StructuredQuery,
)
from langchain.retrievers.self_query.elasticsearch import ElasticsearchTranslator
from langchain_core.documents import Document


def query_to_filter(query: TutorialSearch) -> dict:
    comparisons = []
    if query.earliest_publish_date is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.GTE,
                attribute="publish_date",
                value={"type": "date", "date": query.earliest_publish_date},
            )
        )
    if query.latest_publish_date is not None:
        comparisons.append(
            Comparison(
                comparator=Comparator.LT,
                attribute="publish_date",
                value={"type": "date", "date": query.latest_publish_date},
            )
        )
    if comparisons:
        filter = Operation(operator=Operator.AND, arguments=comparisons)
        return ElasticsearchTranslator().visit_operation(filter)
    else:
        return {}


def content_search(input: dict) -> List[Document]:
    return vectorstore.similarity_search_with_score(
        input["query"].content_search, filter=input["filter"]
    )


def dedup(input: List[List[Tuple[Document, float]]]) -> List[Tuple[Document, float]]:
    """Since document chunk should have a unique (source, start_index) and can be deduped that way."""
    title_and_index = []
    content_docs = []
    for result in input:
        for doc, score in result:
            if (
                doc.metadata["source"],
                doc.metadata["start_index"],
            ) not in title_and_index:
                content_docs.append((doc, score))
                title_and_index.append(
                    (doc.metadata["source"], doc.metadata["start_index"])
                )

    return content_docs


def sort(docs_and_scores: List[Tuple[Document, float]]) -> List[Tuple[Document, float]]:
    """Given our vector store our scores are cosine similarity, so we sort in descending order."""
    return sorted(docs_and_scores, key=(lambda doc_score: doc_score[1]), reverse=True)

In [59]:
from langchain_core.runnables import RunnableLambda


def queries_and_filters(queries: List[TutorialSearch]) -> List[Dict]:
    return [{"query": q, "filter": query_to_filter(q)} for q in queries]


retrieval = (
    query_analyzer_with_examples
    | queries_and_filters
    | RunnableLambda(content_search).map()
    | dedup
    | sort
)

In [ ]:
results = retrieval.invoke(
    "how to use multi-modal models in a chain and turn chain into a rest api"
)

In [61]:
[doc.metadata["title"] for doc, _ in results]

['Build and Deploy a RAG app with Pinecone Serverless',
 'Building a Research Assistant from Scratch',
 'Build and Deploy a RAG app with Pinecone Serverless',
 'Building a web RAG chatbot: using LangChain, Exa (prev. Metaphor), LangSmith, and Hosted Langserve',
 'Streaming Events: Introducing a new `stream_events` method',
 'Getting Started with Multi-Modal LLMs',
 'LangChain Agents with Open Source Models!']

In [ ]:
results = retrieval.invoke("RAG tutorial published in 2023")

In [63]:
[(doc.metadata["title"], doc.metadata["publish_date"]) for doc, _ in results]

[('Getting Started with Multi-Modal LLMs', '2023-12-20'),
 ('LangServe and LangChain Templates Webinar', '2023-11-02'),
 ('Getting Started with Multi-Modal LLMs', '2023-12-20'),
 ('SQL Research Assistant', '2023-12-19')]